Simulate the generation of predctions on the machine and make comparasion between the results. (Simulation not just generation)

Load Libraries

In [ ]:
#### Load Libraries
import numpy as np
from joblib import load
from keras.models import Model, load_model
import glob 
import  pandas as pd
import matplotlib.pyplot as plt
from keras.losses import mean_absolute_error as mae, mean_squared_error as mse, mean_absolute_percentage_error as mape
import os
from pymongo import MongoClient
import plotly.express as ex
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from datetime import datetime
from datetime import timedelta
import plotly.io as pio

Initialize Tensorflow

In [ ]:
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

DB Config

In [ ]:
#####DB Config
host= "DevEdgeV32"
port=27017
all_data= True
get_preprocessed_data= False

Set parameters

In [ ]:
data_path= 'G:/Innovations@HELLER/DN/KI/Temperaturkompensation/New_Experiments/assets/data/_N_072221LOG_PLA3_MPF.csv'
preprocessor_dir='G:/Innovations@HELLER/DN/KI/Temperaturkompensation/New_Experiments/assets/preprocessor/'
preprocessor_name='preprocessor.p'
model_dir='G:/Innovations@HELLER/DN/KI/Temperaturkompensation/New_Experiments/assets/model/'
model_name= 'model.h5'
model_input_Columns=  ['t_bett','t_motor', 't_spindle','M8', 'M121', 'M127', 'M7']
output_variable= 'welle_z'
prediction_variable= 'prediction'
window=10
shift= 1
time_shift= +1

Remove t_bett function

In [ ]:
def remove_t_bett(data: np.ndarray):
    result= data[:, 1:]
    result[:,0]= result[:,0] - data[:,0]
    result[:,1]= result[:,1] - data[:,0]
    return result

In [ ]:
def rounding(signals: np.ndarray):
    return np.round(signals,2)

Load Model and Preprocessor

In [ ]:
### load models
pipeline= load(str(preprocessor_dir+preprocessor_name))
print('preprocessor is loaded successfully')
model= load_model(str(model_dir + model_name), compile= False)
print('Model is loaded successfully')
print (model.summary())

Set time priod needed

In [ ]:
start= datetime.strptime("2022-11-12T20:00:00.000+0000", "%Y-%m-%dT%H:%M:%S.%f+0000")
end= datetime.strptime ("2022-11-13T13:00:00.000+0000", "%Y-%m-%dT%H:%M:%S.%f+0000")

Get Data from database

In [ ]:
## Read raw and preprocessing data
client = MongoClient(host= host, port=port)
db = client.h4ai
event_list = db.modelLogs.find({ "date" : { '$gte' : start, '$lt' : end} }).sort('date', 1)
signals= None
i=0
last_prediction= -500
### Go through events in DB
for event in event_list:
    i+=1
    record_list= event['content']
    keys= None
    ### Go through records in each event
    for record in record_list:
    ##### Get only data points to be predicted
        if not all_data:
            if 'given2model' in record:
                if not record['given2model']:
                    continue
            elif record[prediction_variable] == last_prediction:
                continue
            else:
                last_prediction= record[prediction_variable]
        #####
        if 'given2model' in record:
           record['raw_data']['given2model'] =record['given2model']
        keys= record['raw_data'].keys()
        for item in keys:
            record['raw_data'][item]= [record['raw_data'][item]]
        #print ('before',record)
        record['raw_data']['date']=[record['date'] + timedelta(hours= time_shift)]
        record['raw_data'][output_variable]=[record[output_variable]]
        record['raw_data'][prediction_variable]=[record[prediction_variable]]
        record['raw_data']['DRZ2'][0]= float(record['raw_data']['DRZ2'][0])
        record['raw_data']['t_spindle'][0]= np.round(record['raw_data']['t_spindle'][0], decimals=2)
        record['raw_data']['t_motor']= np.round(float(record['raw_data']['t_motor'][0]), decimals= 2)
        #####Add preprocessed Data##########
        if get_preprocessed_data:
            record['raw_data']['0']= record['preprocessed_data'][0]
            record['raw_data']['1']= record['preprocessed_data'][1]
            record['raw_data']['2']= record['preprocessed_data'][2]
            record['raw_data']['3']= record['preprocessed_data'][3]
            record['raw_data']['4']= record['preprocessed_data'][4]
            record['raw_data']['5']= record['preprocessed_data'][5]
        ####################################
        #print ('After ',record)
        lf_signal_point= pd.DataFrame(record['raw_data'])
        if signals is None:
            signals= lf_signal_point
        else:
            signals= signals.append(lf_signal_point,ignore_index= True)
signals['t_motor']= signals['t_motor'].apply(np.float32)
signals.reset_index(inplace= True)
output=  signals[output_variable]
machine_prediction= signals[prediction_variable]

In [ ]:
signals['M8']= signals['M8'].shift(+3)
signals=signals[4:]
signals.reset_index(inplace= True)

Shift the Z_Welle one step back

In [ ]:
length= len(signals)
start= -1
end= -1
phases=[]
for index, row in signals.iterrows():
    if start<0:
        start= index
        continue
    if row['given2model'] or index == length - 1:
        end= index-1
        new_phase= (start, end)
        phases.append(new_phase)
        start= index
        print(new_phase)
for phase in phases:
    start_idx= phase[0]
    end_idx= phase[1]
    signals['welle_z'][start_idx: end_idx + 1]= signals['welle_z'][end_idx+1]

Calculate the error after the correction

In [ ]:
signals['db_prediction_abs_error']= signals[prediction_variable] -signals[output_variable]
signals['db_prediction_abs_error']= 1000 * signals['db_prediction_abs_error']

Visualize the retreived data

In [ ]:
scatter_mode= 'lines'#'lines'# 'lines+markers'# 'markers'
selected_Columns= ['DRZ2','t_motor', 't_spindle' ,'M8', 'M121', 'M127', 'M7','given2model', prediction_variable, output_variable, 'db_prediction_abs_error']
y_axis_names= selected_Columns
#print(preds)
fig= make_subplots(rows=10,cols=1,shared_xaxes= True, print_grid= True, vertical_spacing=0.02)
##t_bett
fig.add_trace(go.Scatter(x= signals['date'], y= signals['DRZ2'], name='DRZ2', mode= scatter_mode), row= 1, col= 1)
fig.update_yaxes(title_text= 'DRZ2', row= 1, col= 1)
##t_motor
fig.add_trace(go.Scatter(x= signals['date'], y= signals['t_motor'], name='t_motor', mode= scatter_mode), row= 2, col= 1)
fig.update_yaxes(title_text= 't_motor C°', row= 2, col= 1)
##t_spindel
fig.add_trace(go.Scatter(x= signals['date'], y= signals['t_spindle'], name='t_spindel', mode= scatter_mode), row= 3, col= 1)
fig.update_yaxes(title_text= 't_spindel C°', row= 3, col= 1)
##'M8'
fig.add_trace(go.Scatter(x= signals['date'], y= signals['M8'], name='M8', mode= scatter_mode), row= 4, col= 1)
fig.update_yaxes(title_text= 'M8 0/1', row= 4, col= 1)
##'M121'
fig.add_trace(go.Scatter(x= signals['date'], y= signals['M121'], name='M121', mode= scatter_mode), row= 5, col= 1)
fig.update_yaxes(title_text= 'M121 0/1', row= 5, col= 1)
##'M127'
fig.add_trace(go.Scatter(x= signals['date'], y= signals['M127'], name='M127', mode= scatter_mode), row= 6, col= 1)
fig.update_yaxes(title_text= 'M127 0/1', row= 6, col= 1)
##'M7'
fig.add_trace(go.Scatter(x= signals['date'], y= signals['M7'], name='M7', mode= scatter_mode), row= 7, col= 1)
fig.update_yaxes(title_text= 'M7 0/1', row= 7, col= 1)
##'given2model'
fig.add_trace(go.Scatter(x= signals['date'], y= signals['given2model'], name='given2model', mode= scatter_mode), row= 8, col= 1)
fig.update_yaxes(title_text= 'given2model 0/1', row= 8, col= 1)
##Verlagerung
fig.add_trace(go.Scatter(x= signals['date'], y= 1000 * signals[output_variable], name='Gemesene Verlagerung (Welle)', mode= scatter_mode), row= 9, col= 1)
fig.add_trace(go.Scatter(x= signals['date'], y= 1000 * signals[prediction_variable], name='KI-basierte Verlagerung (Welle)', mode= scatter_mode), row= 9, col= 1)
fig.update_yaxes(title_text= 'Verlagerung Micro-Meter', row= 9, col= 1)
##Restfehler
fig.add_trace(go.Scatter(x= signals['date'], y= signals['db_prediction_abs_error'], name=' Restfehler', mode= scatter_mode),  row= 10 , col= 1)
## Draw the tolerence +-5
fig.add_trace(go.Scatter(x= signals['date'], y= np.full_like(signals['db_prediction_abs_error'],5), name='+5 Obere Grenze', mode= scatter_mode),  row= 10 , col= 1)
fig.add_trace(go.Scatter(x= signals['date'], y= np.full_like(signals['db_prediction_abs_error'],-5), name='- 5 Obere Grenze', mode= scatter_mode),  row= 10 , col= 1)
fig.update_yaxes(title_text= output_variable, row= 10, col= 1)
fig.update_layout(height=1200, width=1400, title_text= 'M57002 Machine Data')
fig.show()

Initialize model input

In [ ]:
rough_signals= signals[model_input_Columns].to_numpy()

Simulate the prediction by passing values with 5 seconds

In [ ]:
##### Simulate the prediction by passing values with 5 seconds ########
preds= []
#selected_Columns=  ['0','1','2','3','4','5']
input_data= signals
shape=(1,window,6)
first_run= True
# signal liste für preprocessing erstellen (Reihenfolge wie in input_signals im config-File)
signaloffsetnumber = 0
inputsignal_list= model_input_Columns
recording_frequency= 120
# 2 Minuten-Problem
time_point = input_data['date'][0]#datetime.utcnow()
time_point_dbwait = input_data['date'][0]#datetime.utcnow()
under_2_min = True
dbwrite_wait = True
go_on_prog = True
hlpvar = False
prev_input= None
before_prev_input= prev_input
out=0
predicted= False
for index, data_point in input_data.iterrows():
    predicted= False
    #print ('index: ', index, ', point: ', data_point)
    meas_trigger= data_point['given2model']
    current_time_2min = data_point['date']
    under_2_min = (current_time_2min - time_point).total_seconds() <= recording_frequency  # 120 #120
    #if meas_trigger:
        # ### correct last value of prediction
        # # 1- Correct cooling values in previous time step t-1 to be as t-2
        # signals.loc[index - 1,'M8']= before_prev_input['M8']
        # signals.loc[index - 1,'M121']= before_prev_input['M121']
        # signals.loc[index - 1,'M127']= before_prev_input['M127']
        # signals.loc[index - 1,'M7']= before_prev_input['M7']
        # # 1- Correct cooling values of the current time step t to be as t-2
        # signals.loc[index ,'M8']= before_prev_input['M8']
        # signals.loc[index ,'M121']= before_prev_input['M121']
        # signals.loc[index ,'M127']= before_prev_input['M127']
        # signals.loc[index ,'M7']= before_prev_input['M7']
        # data_point[['M8', 'M121', 'M127', 'M7']]= before_prev_input[['M8', 'M121', 'M127', 'M7']]
        #print ('index: ', index, ', updated point: ', data_point)
    new_signal_list = [float(data_point[sig]) for sig in inputsignal_list]
    #print ('New_signal_list: ', new_signal_list)
    signals_ = pipeline.transform(np.asarray([new_signal_list]))#np.asarray([new_signal_list])#
    #current_time_dbwait = datetime.utcnow()
    #dbwrite_wait = (current_time_dbwait - time_point_dbwait).total_seconds() <= 120 #120
    if first_run:
        # Auffüllen der Struktur - Bei erstem Start ist nur ein Datensatz vorhanden,
        # dieser wird mit np.full in alle kopiert
        array = np.full(shape, signals_[0])
        array = np.float32(array)
        #outputs = ort_sess.run(None, {'x': array})
        outputs =model.predict(array)
        #out = outputs[0][0][0] / 1000
        out = outputs[0][0] / 1000
        #print('Predciction First time. Input Buffer: ', array, 'prediction: ', out)
        predicted= True
        preds.append(out)
        first_run = False
        continue
    elif under_2_min:
        #print('under2min 2: ')
        #print('Buffer new state in under2min 2: ', array)
        ### Just add last prediction
        if meas_trigger: #not dbwrite_wait:
            #print("....durchshiften Pos1")
            #print('Buffer Current state before Shift: ', array)
            array[0][:-1] = array[0][1:]
            array[0, -1, :] = signals_[0]
            #print('Buffer new  state After Shift: ', array)
            #print ('under 2 min, meas_trigger True')
    else:
        #print ('under 2 min False')
        # durchshiften der gelesenen Daten im array - FiFo
        if meas_trigger: #not dbwrite_wait:
           # print ('under 2 min False, meas_trigger True')
            #print("....durchshiften Pos2")
           # print('Buffer Current state before Shift: ', array)
            array[0][:-1] = array[0][1:]
            array[0, -1, :] = signals_[0]
            #print('Buffer new  state After Shift: ', array)
            #time_point_dbwait = current_time_dbwait
            #hlpvar = True
        time_point = current_time_2min
        #time_point_dbwait = current_time_dbwait
    
    if not (under_2_min and go_on_prog) or hlpvar or meas_trigger:
        # if show_debugprints:
            # print(f"current input data: {array}")
        if meas_trigger: #hlpvar:
            array = np.float32(array)
            # entfernt, und nach ober vor while verschoben 17.06.22/gst ort_sess = ort.InferenceSession(model)
            # Aufrufen der inference-Funktion von Mustafa.d.h hier wird aus den Werten die prediction erzeugt
            outputs =model.predict(array)
            #out = outputs[0][0][0] / 1000
            out = outputs[0][0] / 1000
            #print('Prediction Normal state. Input Buffer: ', array, 'prediction: ', out)
            predicted= True
            preds.append(out)
    if not predicted:
        preds.append(out)
    before_prev_input= prev_input    
    prev_input= data_point
    #print('#########################################################################')

In [ ]:
#signals[output_variable]= signals[output_variable].shift(+1)# + 0.001441305# +2.54* 0.001

Assign local prediction to the main dataframe

In [ ]:
len(preds)

In [ ]:
signals['local_preds']= preds
local_prediction_error= signals[output_variable] - signals['local_preds']

Visualize the results

In [ ]:
images_path='//Heller.biz/hnt/Steuerungstechnik/Innovations@HELLER/DN/KI/Temperaturkompensation/2021_Spindelwachstumskompensation_KI_HSU_SC63/Messungen DC100 H5000 M57002/csvs/smoothed/AI_Model/Backup/Z_Welle/v2.7.0/five_seconds/'
## Comparasion between two sources:
from turtle import color
scatter_mode= 'lines'#'lines'# 'lines+markers'# 'markers'
selected_Columns= ['t_bett','t_motor', 't_spindle' ,'M8', 'M121', 'M127', 'M7','given2model', prediction_variable]
y_axis_names= selected_Columns
#print(preds)
fig= make_subplots(rows=len(selected_Columns)+1,cols=1,shared_xaxes= True, print_grid= True, subplot_titles= selected_Columns, vertical_spacing=0.02)
for i in range(len(selected_Columns)):
    fig.add_trace(go.Scatter(x= signals['date'], y= signals[selected_Columns[i]], name=selected_Columns[i], mode= scatter_mode), row= i+1, col= 1,)
    fig.update_yaxes(title_text= y_axis_names[i], row= i+1, col= 1)
##Draw the prediciton and the real values of displacement on Welle
fig.add_trace(go.Scatter(x= signals['date'],y= signals[output_variable], name=output_variable, mode= scatter_mode),  row= len(selected_Columns) , col= 1)
fig.add_trace(go.Scatter(x= signals['date'], y=  signals['local_preds'], name='Local predictions', mode= scatter_mode),  row= len(selected_Columns) , col= 1)
fig.update_yaxes(title_text= output_variable, row= len(selected_Columns), col= 1)
fig.add_trace(go.Scatter(x= signals['date'], y=   1000*local_prediction_error, name='New Model prediction Error', mode= scatter_mode),  row= len(selected_Columns)+1 , col= 1)
## Draw the tolerence +-5
fig.add_trace(go.Scatter(x= signals['date'], y= np.full_like(local_prediction_error,5), name='+5 Max Error', mode= scatter_mode),  row= len(selected_Columns)+1 , col= 1)
fig.add_trace(go.Scatter(x= signals['date'], y= np.full_like(local_prediction_error,-5), name='-5 Min Error', mode= scatter_mode),  row= len(selected_Columns)+1 , col= 1)

#fig.add_trace(go.Scatter(x= signals['date'], y=   1000*( np.abs(signals[output_variable] - signals[prediction_variable])), name='Old Model prediction Error', mode= scatter_mode),  row= len(selected_Columns)+1 , col= 1)
fig.update_yaxes(title_text= 'Error (Micro-meter)', row= len(selected_Columns)+1, col= 1)
fig.update_layout(height=1200, width=1400, title_text= 'Prediction Results')
#pio.write_image(fig, str(images_path +'versuch_20_09_iso.png'), format='png')
fig.show()
